In [1]:
from eeva import utils
from eeva.utils import Prompts, Model
from eeva.interview import Interviewer, Analyst
from eeva.simulation import Character, InterviewSimulation

from pathlib import Path

In [2]:
utils.load_secrets("secrets.json")
output_dir = Path("output")
prompts = Prompts("prompts")

In [3]:
model = Model(
    model_name="gpt-4o-mini",
    model_provider="openai",
)

In [4]:
interviewer = Interviewer(
    system_prompt=prompts.get("default/life-story-interviewer"),
    model=model,
)

In [5]:
character = Character(
    name="Hermione",
    system_prompt=prompts.get("default/hermione-basic"),
    model=model,
)

In [6]:
interview = InterviewSimulation.initialize(
    interviewer=interviewer,
    character=character,
)

In [ ]:
interview.advance(3)
interview.print_messages()

In [8]:
interview.save_to_file(output_dir / "interview.json")

In [9]:
analyst = Analyst(
    system_prompt=prompts.get("default/freudian-analyst-system"),
    instruction=prompts.get("default/unhingedness-analyst-instruction"),
    model=model,
)

In [ ]:
print(interview.analyze(analyst).pretty_format())